In [1]:

import time
import pandas as pd
import re

import requests
import json

import mysql
import mysql.connector
import os

import datetime
import unicodedata

from dotenv import load_dotenv
load_dotenv(".env")

True

In [ ]:

def get_cursor():
    # MySQLに接続
    conn = mysql.connector.connect(
        host=os.environ.get("AWS_SLOMAP_RDS_HOST"),
        user=os.environ.get("AWS_SLOMAP_RDS_USER"),
        password=os.environ.get("AWS_SLOMAP_RDS_PASSWORD"),
        database=os.environ.get("AWS_SLOMAP_RDS_DATABASE"),
    )
    return conn

conn = get_cursor()
# カーソルを取得
cursor = conn.cursor()
